# Extracting and Uploading MaStR - Winddata with open-mastr

In [ ]:
from open_mastr import Mastr
import pandas as pd
import geopandas as gpd

In [ ]:
tables_list = ["wind", "location", "permit"]
db = Mastr()
db.download(method = "bulk",
            data=tables_list)

In [ ]:
### Inspect db
conn = db.engine # Connection engine
tables = pd.read_sql_query('SELECT name from sqlite_master where type= "table";', conn)
df_wind = pd.read_sql_table("wind_extended", conn)
columns_wind = list(df_wind.columns)

In [46]:
### Subset to Land
# df_wind = df_wind[df_wind.Lage == "Windkraft an Land"]
df_wind.Kuestenentfernung.describe() 
df_wind.Kuestenentfernung.unique()  # Only for Offshore
### Remove columns
print(columns_wind)

Index(['NameWindpark', 'Lage', 'Seelage', 'ClusterOstsee', 'ClusterNordsee',
       'Hersteller', 'HerstellerId', 'Technologie', 'Typenbezeichnung',
       'Nabenhoehe', 'Rotordurchmesser', 'Rotorblattenteisungssystem',
       'AuflageAbschaltungLeistungsbegrenzung',
       'AuflagenAbschaltungSchallimmissionsschutzNachts',
       'AuflagenAbschaltungSchallimmissionsschutzTagsueber',
       'AuflagenAbschaltungSchattenwurf', 'AuflagenAbschaltungTierschutz',
       'AuflagenAbschaltungEiswurf', 'AuflagenAbschaltungSonstige',
       'Wassertiefe', 'Kuestenentfernung', 'Buergerenergie',
       'Nachtkennzeichen', 'EegMastrNummer', 'NetzbetreiberMastrNummer',
       'Registrierungsdatum', 'EinheitMastrNummer',
       'DatumLetzteAktualisierung', 'LokationMastrNummer',
       'NetzbetreiberpruefungStatus', 'NetzbetreiberpruefungDatum',
       'AnlagenbetreiberMastrNummer', 'Land', 'Bundesland', 'Landkreis',
       'Gemeinde', 'Gemeindeschluessel', 'Postleitzahl', 'Gemarkung',
       'FlurFl

In [37]:
### Very few plants actually have an address -> Not useful for displaying?
df_address = df_wind[["Strasse", "StrasseNichtGefunden",
       "Hausnummer", "HausnummerNichtGefunden", "Adresszusatz"]]
df_address.info()

In [48]:
columns_remove = ["Lage",	"Seelage",	"ClusterOstsee", "ClusterNordsee", "Strasse", "StrasseNichtGefunden",
       "Hausnummer", "HausnummerNichtGefunden", "Adresszusatz", "NetzbetreiberpruefungStatus", 
       "NetzbetreiberpruefungDatum", "Wassertiefe", "Kuestenentfernung"]
columns_wind = [col for col in columns_wind if col not in columns_remove]
df_wind = df_wind[columns_wind]

In [52]:
geo_columns = [ 'Laengengrad',
 'Breitengrad',
 'UtmZonenwert',
 'UtmEast',
 'UtmNorth',
 'GaussKruegerHoch',
 'GaussKruegerRechts']
df_geo = df_wind[geo_columns]
df_geo.info()
df_geo.UtmZonenwert.unique()

<class 'pandas.core.frame.DataFrame'>
Index: 33440 entries, 0 to 35226
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Laengengrad         32494 non-null  float64
 1   Breitengrad         32494 non-null  float64
 2   UtmZonenwert        0 non-null      object 
 3   UtmEast             0 non-null      float64
 4   UtmNorth            0 non-null      float64
 5   GaussKruegerHoch    0 non-null      float64
 6   GaussKruegerRechts  0 non-null      float64
dtypes: float64(6), object(1)
memory usage: 2.0+ MB


array([None], dtype=object)